(If you are missing any of the required packages, run the cell below to install them)

In [ ]:
pip install numpy pandas gensim spacy nltk pyLDAvis

In [1]:
import numpy as np
import pandas as pd

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy and nltk
import spacy
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

#visualizations
import pyLDAvis
import pyLDAvis.gensim

/Users/nathanielnichols/Documents/Python_Projects/TikTokProject/tiktokvenv/lib/python3.11/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nathanielnichols/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
total_df = pd.read_csv('output_data/tiktok_data.csv')
total_df

,Unnamed: 0,influencer_name,vid_ID,language,caption,caption_translated,caption_sentiment,audio_original,audio_translated,audio_sentiment,screen_text,text_translated,text_sentiment,duration,color_dominance,facial_presence,video_link
0,0,Elyse_Myers,10,english,no title found,no title found,-0.2960,Travis Kelce winning the Super Bowl in 2023 in...,Travis Kelce winning the Super Bowl in 2023 in...,0.9903,Tm sorry but I just saw anedit Tm sorry but ...,Tm sorry but I just saw anedit Tm sorry but ...,0.9998,0:00:21,"(122, 100, 94)",0.616000,https://www.tiktok.com/@elysemyers/video/73347...
1,1,Nadia_Addesi,100,english,The struggle is real 🥲 ib @hollymtaylorlpc . ...,The struggle is real 🥲 ib @hollymtaylorlpc . ...,-0.9062,no text,no text,-0.2960,When re a therapist and you realize how ...,When re a therapist and you realize how ...,-0.0654,0:00:11,"(142, 132, 125)",0.318182,https://www.tiktok.com/@nadiaaddesi/video/7266...
2,2,Laila_Algohary,1002,arabic,NaN,into,0.0000,no text,no text,-0.2960,Iamaanadakhla Belm nafs Ana Lmael Zaramy v...,Iamaanadakhla Belm Nafs ana lmael zaramy veegy...,0.0000,0:00:08,"(87, 79, 70)",0.540000,https://www.tiktok.com/@thetherapyroomm/video/...
3,3,Laila_Algohary,1006,arabic,NaN,into,0.0000,no text,no text,-0.2960,lst therapy session 1st therapy session ls...,lst therapy session 1st therapy session ls...,0.0000,0:00:15,"(87, 79, 71)",0.736264,https://www.tiktok.com/@thetherapyroomm/video/...
4,4,Laila_Algohary,1009,arabic,NaN,into,0.0000,انا جاي لي مسج كثيره من الناس بتتكلم على ان هم...,I come to me a lot of people who speak that th...,0.9615,NaN,into,0.0000,0:02:41,"(101, 84, 70)",0.990674,https://www.tiktok.com/@thetherapyroomm/video/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679,679,Nadia_Addesi,99,english,Do you have an anxious attachment style or are...,Do you have an anxious attachment style or are...,0.5106,no text,no text,-0.2960,are VoU mad at me? Hey; No: Why would be...,are VoU mad at me? Hey; No: Why would be...,0.8652,0:00:18,"(131, 124, 116)",0.943396,https://www.tiktok.com/@nadiaaddesi/video/7286...
680,680,Laila_Algohary,991,arabic,بلاش تكون #ضحية,Blash is a victim,-0.2732,اسعارهم كثير جدا بتبقى عاليه فالناس مش بتعرف ت...,"Their prices are very high, so people do not k...",0.8592,List your reasons here The high prices LiS...,List your reasons here The high prices LiS...,0.0000,0:00:58,"(149, 126, 115)",0.280802,https://www.tiktok.com/@thetherapyroomm/video/...
681,681,Laila_Algohary,994,arabic,#psychology #psychologistoftiktok #therapy #le...,#psychology #psychologistoftiktok #therapy #le...,0.0000,انا عارفه انا عندي ايه وعارفه اني محتاجه مساعد...,"I know what I have, and I know that I need hel...",0.9044,TkTok @ thetherapvroon TkTok @ tnetherap...,TKTOK @ Theetraphvroon Tktok @ TNETHERAPROO B ...,0.9966,0:01:11,"(87, 55, 77)",0.950935,https://www.tiktok.com/@thetherapyroomm/video/...
682,682,Laila_Algohary,996,arabic,NaN,into,0.0000,no text,no text,-0.2960,0 TkTok @ thetherapyroomm 06 TkTok @...,0 TkTok @ thetherapyroomm 06 TkTok @...,0.0000,0:00:15,"(96, 44, 73)",0.011111,https://www.tiktok.com/@thetherapyroomm/video/...


In [3]:
# Lemmatization
def lemmatization(texts, allowed_posttags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    texts_out = []
    for text in texts:
        if not(text == 'no text'):
            doc = nlp(str(text))
            new_text = []
            for token in doc:
                if token.pos_ in allowed_posttags and not(token.lemma_ in stopwords.words('english')):
                    new_text.append(token.lemma_)
            final = " ".join(new_text)
            texts_out.append(final)

    return texts_out

In [4]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return final

In [6]:
# Audio topic model for entire dataset

df_list = total_df['audio_translated'].tolist()

lemmatized_texts = lemmatization(df_list)

data_words = gen_words(lemmatized_texts)

# Bigrams and Trigrams
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=20)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=20)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

# TF - IDF Removal
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.09
words = []
words_missing_in_tfidf = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=25,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto')

pyLDAvis.enable_notebook()
# Change the R parameter below to change the number of relevant words shown
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds', R=20)
print("Topics: Audio from Entire Dataset")
vis

Topics: Audio from Entire Dataset


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
10     0.083744 -0.402785       1        1  7.764970
21     0.141536  0.383220       2        1  7.601181
22    -0.402921  0.022398       3        1  6.271476
19    -0.315432 -0.264428       4        1  6.084779
20    -0.314167  0.329801       5        1  5.660916
24    -0.087580  0.360660       6        1  5.250607
2      0.212596 -0.273720       7        1  5.212812
13     0.348581 -0.175579       8        1  5.091496
17     0.351625 -0.009016       9        1  4.837838
18     0.292909  0.243773      10        1  4.593254
8      0.223037 -0.020523      11        1  4.203109
14     0.255234  0.120506      12        1  4.097090
0     -0.254890  0.158587      13        1  4.094649
23    -0.298886 -0.081760      14        1  4.004067
6     -0.101778 -0.339619      15        1  3.629708
11    -0.030878 -0.195296      16        1  3.484286
7     -0.162931 -0.200293      17        1  3.341641
15    -0.156482  0.012590      18        1  3.051847
16     0.017369  0.244184      19        1  2.523306
4      0.101174 -0.159097      20        1  1.989693
5      0.073923 -0.033334      21        1  1.811878
1     -0.006948  0.105622      22        1  1.562883
9      0.015320  0.010537      23        1  1.409158
3     -0.063371  0.064167      24        1  1.397448
12     0.079216  0.099404      25        1  1.029908, topic_info=         Term        Freq       Total Category  logprob  loglift
40       need  223.000000  223.000000  Default  20.0000    20.00
44     people  193.000000  193.000000  Default  19.0000    19.00
154    person  190.000000  190.000000  Default  18.0000    18.00
519     think  165.000000  165.000000  Default  17.0000    17.00
286      life  125.000000  125.000000  Default  16.0000    16.00
...       ...         ...         ...      ...      ...      ...
1507    party    0.026259    1.168777  Topic25  -8.8394     0.78
2098      joy    0.026259    1.168777  Topic25  -8.8394     0.78
2275  respect    0.026259    1.168777  Topic25  -8.8394     0.78
1832   mental    0.026259    1.168778  Topic25  -8.8394     0.78
1643   mother    0.026259    1.168783  Topic25  -8.8394     0.78

[637 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
653       6  0.304852    ability
653      16  0.508086    ability
246       5  0.915145       able
246       9  0.069068       able
247       4  0.969174     accept
...     ...       ...        ...
12        9  0.979553       year
750      11  0.907672  yesterday
1597     10  0.319171        yet
1597     21  0.478756        yet
901      12  0.499839      youth

[690 rows x 3 columns], R=20, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 22, 23, 20, 21, 25, 3, 14, 18, 19, 9, 15, 1, 24, 7, 12, 8, 16, 17, 5, 6, 2, 10, 4, 13])

In [16]:
# Audio topic model for Egyptian videos

df_list = total_df[total_df['language'] == 'arabic']['audio_translated'].tolist()

lemmatized_texts = lemmatization(df_list)

data_words = gen_words(lemmatized_texts)

# Bigrams and Trigrams
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=20)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=20)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

# TF - IDF Removal
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.09
words = []
words_missing_in_tfidf = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=19,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto')

pyLDAvis.enable_notebook()
# Change the R parameter below to change the number of relevant words shown
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds', R=20)
print("Topics: Audio from Egyptian Videos")
vis

Topics: Audio from Egyptian Videos


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.345518  0.152048       1        1  12.823328
6      0.090072 -0.300251       2        1   9.043951
14    -0.281112 -0.198314       3        1   8.108916
5     -0.084438  0.305949       4        1   7.883211
8      0.223838 -0.214712       5        1   7.494649
16     0.303686 -0.059634       6        1   7.161872
11     0.086829  0.256970       7        1   5.835654
1     -0.100681 -0.264991       8        1   5.463891
15    -0.277050  0.073999       9        1   5.448684
2      0.136629  0.144706      10        1   4.084529
12     0.154700  0.000388      11        1   4.021838
4     -0.022677  0.183450      12        1   3.822435
18    -0.216010 -0.069852      13        1   3.593368
10    -0.191871  0.172373      14        1   3.592138
7     -0.028010 -0.157079      15        1   3.067924
3     -0.095406  0.066407      16        1   2.656303
13     0.055224 -0.062793      17        1   2.508721
9     -0.100832 -0.035897      18        1   1.722181
17     0.001592  0.007233      19        1   1.666409, topic_info=         Term       Freq      Total Category  logprob  loglift
332      life  39.000000  39.000000  Default  20.0000  20.0000
27       need  79.000000  79.000000  Default  19.0000  19.0000
89    feeling  23.000000  23.000000  Default  18.0000  18.0000
20       know  41.000000  41.000000  Default  17.0000  17.0000
173      love  41.000000  41.000000  Default  16.0000  16.0000
..        ...        ...        ...      ...      ...      ...
673  material   0.383931   3.086433  Topic19  -5.0631   2.0102
219      body   0.739306   6.333411  Topic19  -4.4078   1.9466
660      sure   0.739308   7.637664  Topic19  -4.4078   1.7594
214     video   0.739308   8.320735  Topic19  -4.4078   1.6737
57    disease   0.739308   9.606587  Topic19  -4.4078   1.5300

[553 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
490      12  0.813096  ability
234       2  0.074306     able
234       9  0.520142     able
234      11  0.371530     able
235       2  0.939385   accept
...     ...       ...      ...
158      13  0.492862     year
158      14  0.369647     year
158      15  0.123216     year
993      11  0.540253     yolk
994      11  0.772522    young

[730 rows x 3 columns], R=20, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 7, 15, 6, 9, 17, 12, 2, 16, 3, 13, 5, 19, 11, 8, 4, 14, 10, 18])

In [23]:
# Audio topic model for Brazilian videos

df_list = total_df[total_df['language'] == 'portuguese']['audio_translated'].tolist()

lemmatized_texts = lemmatization(df_list)

data_words = gen_words(lemmatized_texts)

# Bigrams and Trigrams
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=20)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=20)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

# TF - IDF Removal
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.09
words = []
words_missing_in_tfidf = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=19,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto')

pyLDAvis.enable_notebook()
# Change the R parameter below to change the number of relevant words shown
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds', R=20)
print("Topics: Audio from Brazilian Videos")
vis

Topics: Audio from Brazilian Videos


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11     0.199260 -0.317148       1        1  14.544665
4      0.372682 -0.072998       2        1  10.761135
10     0.144841  0.313783       3        1   8.017330
13     0.239844 -0.091674       4        1   7.775774
1      0.291975  0.158629       5        1   7.315078
3     -0.298243  0.152995       6        1   5.887316
0     -0.316071 -0.084359       7        1   5.670444
9      0.042634 -0.244909       8        1   5.083372
2     -0.111255 -0.281864       9        1   4.771392
15    -0.201386 -0.169431      10        1   4.234548
17    -0.221869  0.023188      11        1   3.850306
18    -0.060748  0.268739      12        1   3.824825
6     -0.051315 -0.136004      13        1   3.507300
16     0.084229 -0.045064      14        1   3.481754
5     -0.141973  0.200921      15        1   3.056907
8      0.115537  0.090824      16        1   2.879556
14    -0.088770  0.058283      17        1   2.344752
7      0.023595  0.151626      18        1   2.152953
12    -0.022966  0.024461      19        1   0.840592, topic_info=         Term       Freq      Total Category  logprob  loglift
379    person  73.000000  73.000000  Default  20.0000  20.0000
46       want  92.000000  92.000000  Default  19.0000  19.0000
17       life  41.000000  41.000000  Default  18.0000  18.0000
28     people  68.000000  68.000000  Default  17.0000  17.0000
708    mother  40.000000  40.000000  Default  16.0000  16.0000
..        ...        ...        ...      ...      ...      ...
66        eye   0.294395   7.332301  Topic19  -5.3217   1.5637
21     minute   0.294391   4.479025  Topic19  -5.3217   2.0566
61   dopamine   0.294389   3.638394  Topic19  -5.3217   2.2644
73    imagine   0.294388   4.048361  Topic19  -5.3217   2.1577
91       sign   0.294386   4.786535  Topic19  -5.3217   1.9901

[596 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
680       4  0.851955      able
680      15  0.070996      able
1013     18  0.754173  absolute
1187      2  0.236308    accept
1187      9  0.590769    accept
...     ...       ...       ...
937      17  0.371463       yet
96       12  0.358124     yield
96       19  0.358124     yield
1259      4  0.899856   youtube
1012      9  0.716041    zombie

[857 rows x 3 columns], R=20, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 5, 11, 14, 2, 4, 1, 10, 3, 16, 18, 19, 7, 17, 6, 9, 15, 8, 13])

In [25]:
# Audio topic model for American videos

df_list = total_df[total_df['language'] == 'english']['audio_translated'].tolist()

lemmatized_texts = lemmatization(df_list)

data_words = gen_words(lemmatized_texts)

# Bigrams and Trigrams
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=20)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=20)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

# TF - IDF Removal
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.09
words = []
words_missing_in_tfidf = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=22,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto')

pyLDAvis.enable_notebook()
# Change the R parameter below to change the number of relevant words shown
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds', R=20)
print("Topics: Audio from American Videos")
vis

Topics: Audio from American Videos


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14     0.276810  0.258644       1        1  12.652849
15     0.156401 -0.317591       2        1  10.861795
1     -0.343071 -0.020810       3        1   6.934152
9     -0.106328  0.295778       4        1   6.673496
7      0.229852 -0.185694       5        1   6.227404
4      0.288002 -0.067757       6        1   5.603677
19     0.065853  0.296879       7        1   5.391095
16    -0.155944 -0.265043       8        1   4.292105
13    -0.261594  0.109049       9        1   4.290285
21     0.198511  0.133715      10        1   4.077644
3      0.226806  0.016219      11        1   3.782438
18     0.000046  0.197782      12        1   3.522727
10    -0.186877 -0.165895      13        1   3.386871
20    -0.147722  0.168224      14        1   3.315020
8     -0.012061 -0.251275      15        1   2.990183
6     -0.141429  0.066395      16        1   2.984194
11     0.019632 -0.166022      17        1   2.808242
5     -0.194818 -0.053978      18        1   2.643331
12     0.012741  0.048165      19        1   2.577026
2      0.090716  0.014372      20        1   2.311024
17    -0.055691 -0.046193      21        1   1.436545
0      0.040164 -0.064963      22        1   1.237895, topic_info=        Term       Freq      Total Category  logprob  loglift
1057  number  53.000000  53.000000  Default  20.0000  20.0000
28      feel  60.000000  60.000000  Default  19.0000  19.0000
243      say  34.000000  34.000000  Default  18.0000  18.0000
493     back  27.000000  27.000000  Default  17.0000  17.0000
181       go  41.000000  41.000000  Default  16.0000  16.0000
...      ...        ...        ...      ...      ...      ...
658    check   0.701382   2.808711  Topic22  -4.8113   3.0043
1047  decide   0.701380   3.385991  Topic22  -4.8113   2.8174
262     take   1.277605  19.280830  Topic22  -4.2116   1.6776
772     save   0.552355   4.585722  Topic22  -5.0502   2.2752
1303   smart   0.361290   4.226573  Topic22  -5.4747   1.9323

[685 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
652       7  0.916985  absolutely
326      16  0.884751     account
446       5  0.223214         act
446       6  0.223214         act
446       8  0.446429         act
...     ...       ...         ...
549       1  0.945573       wound
325      13  0.799535       wrong
12        7  0.893742        year
12        8  0.089374        year
1611     10  0.811350         zap

[971 rows x 3 columns], R=20, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 16, 2, 10, 8, 5, 20, 17, 14, 22, 4, 19, 11, 21, 9, 7, 12, 6, 13, 3, 18, 1])

In [28]:
# Text topic model for total dataset

df_list = total_df['text_translated'].tolist()

lemmatized_texts = lemmatization(df_list)

data_words = gen_words(lemmatized_texts)

# Bigrams and Trigrams
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

# TF - IDF Removal
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.09
words = []
words_missing_in_tfidf = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=23,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto')

pyLDAvis.enable_notebook()
# Change the R parameter below to change the number of relevant words shown
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds', R=20)
print("Topics: Audio from Entire Dataset")
vis

Topics: Audio from Entire Dataset


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
19    -0.349304  0.142851       1        1  7.342898
7     -0.258666 -0.214874       2        1  6.991399
12    -0.111700 -0.315769       3        1  6.777841
15     0.232186 -0.239147       4        1  6.550687
3      0.083937 -0.326241       5        1  6.033587
20     0.137585  0.286480       6        1  5.889963
22    -0.307031 -0.034188       7        1  5.057015
10     0.030787  0.222549       8        1  4.985700
0     -0.201993  0.281223       9        1  4.900389
11    -0.051412  0.302036      10        1  4.697448
17     0.097070 -0.102528      11        1  4.671651
6      0.335281 -0.088948      12        1  4.351546
16     0.246796  0.020658      13        1  4.320172
4      0.133947  0.145839      14        1  3.652327
14     0.154844 -0.010314      15        1  3.386003
13     0.257397  0.144274      16        1  3.266406
9     -0.108373  0.110059      17        1  3.191515
5     -0.144280  0.014506      18        1  2.999373
21    -0.004747 -0.198855      19        1  2.861044
8     -0.042564 -0.109728      20        1  2.757369
1     -0.137711 -0.086742      21        1  2.477643
2      0.008791  0.004734      22        1  1.472513
18    -0.000842  0.052125      23        1  1.365511, topic_info=                       Term         Freq        Total Category  logprob  \
2262                   talk  3398.000000  3398.000000  Default  20.0000   
314                    need  2583.000000  2583.000000  Default  19.0000   
313                    know  2324.000000  2324.000000  Default  18.0000   
2024                    let  2436.000000  2436.000000  Default  17.0000   
87                   people  2158.000000  2158.000000  Default  16.0000   
...                     ...          ...          ...      ...      ...   
1149  exactlyuhere_theywant     1.704436     3.084062  Topic23  -7.0707   
1194                tiggere     1.704436     3.084062  Topic23  -7.0707   
1195                   tobe     1.704436     3.084062  Topic23  -7.0707   
3424                   heal    33.940713   100.071335  Topic23  -4.0793   
1141                   calm     1.982981    16.218296  Topic23  -6.9193   

      loglift  
2262  20.0000  
314   19.0000  
313   18.0000  
2024  17.0000  
87    16.0000  
...       ...  
1149   3.7006  
1194   3.7006  
1195   3.7006  
3424   3.2124  
1141   2.1921  

[539 rows x 6 columns], token_table=       Topic      Freq                                  Term
term                                                        
4897      10  0.130806                                  aauz
4897      14  0.752132                                  aauz
4897      17  0.098104                                  aauz
11125     17  0.965257                                  aavz
11127     14  0.975997                                  abeh
...      ...       ...                                   ...
3082       6  0.990793                                 world
4884      11  0.979935                                 wound
8397       8  0.850846  wth_thelevelof_independenceand_space
27        11  0.979733                                  year
2692      23  0.903228                                   you

[531 rows x 3 columns], R=20, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[20, 8, 13, 16, 4, 21, 23, 11, 1, 12, 18, 7, 17, 5, 15, 14, 10, 6, 22, 9, 2, 3, 19])

In [31]:
# Text topic model for Egyptian videos

df_list = total_df[total_df['language'] == 'arabic']['text_translated'].tolist()

lemmatized_texts = lemmatization(df_list)

data_words = gen_words(lemmatized_texts)

# Bigrams and Trigrams
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

# TF - IDF Removal
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.09
words = []
words_missing_in_tfidf = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=16,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto')

pyLDAvis.enable_notebook()
# Change the R parameter below to change the number of relevant words shown
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds', R=20)
print("Topics: Audio from Egyptian Videos")
vis

Topics: Audio from Egyptian Videos


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.484064  0.135273       1        1  25.413783
3      0.255952 -0.279535       2        1   8.042443
10     0.387668 -0.107898       3        1   7.821125
6      0.365560  0.142072       4        1   6.444859
1      0.206198  0.349823       5        1   6.394149
8      0.061319 -0.333040       6        1   6.152998
11    -0.120249 -0.371704       7        1   5.553259
9     -0.111628 -0.172886       8        1   5.104040
12    -0.210292 -0.001556       9        1   4.849122
2     -0.219534  0.256562      10        1   4.650591
7      0.209768  0.052379      11        1   4.442121
15     0.083991 -0.096864      12        1   3.819362
14     0.079332  0.219143      13        1   3.691628
0     -0.057228  0.094615      14        1   2.859647
13    -0.057058  0.329158      15        1   2.476114
5     -0.389734 -0.215539      16        1   2.284757, topic_info=                 Term         Freq        Total Category  logprob  loglift
2980             talk  2806.000000  2806.000000  Default  20.0000  20.0000
2103         medicine  2485.000000  2485.000000  Default  19.0000  19.0000
2070              let  2296.000000  2296.000000  Default  18.0000  18.0000
1661         simplify  1175.000000  1175.000000  Default  17.0000  17.0000
3836               es  1971.000000  1971.000000  Default  16.0000  16.0000
...               ...          ...          ...      ...      ...      ...
6797   thetherapvoomش    27.608862    29.015335  Topic16  -3.6869   3.7292
1608             hair    24.958894    26.365534  Topic16  -3.7878   3.7241
6789  theetrapvrooomm    19.725038    21.131510  Topic16  -4.0232   3.7100
6793    thetherapvoom    19.725038    21.131510  Topic16  -4.0232   3.7100
527             topic    33.199898   254.973441  Topic16  -3.5025   1.7403

[405 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
4655     14  0.920924         aais
1037      6  0.921172         able
6400     10  0.981033       accept
1769     11  0.855463    accompany
3648      2  0.248344         adhd
...     ...       ...          ...
5456     12  0.815624           قش
3166     13  0.944108  للقلق_طبيعي
5465     12  0.815624    مقش_مقشاق
5466     12  0.859210        مقشاق
5467     12  0.626859  مقشاق_مقشاق

[415 rows x 3 columns], R=20, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 11, 7, 2, 9, 12, 10, 13, 3, 8, 16, 15, 1, 14, 6])

In [35]:
# Text topic model for Brazilian videos

df_list = total_df[total_df['language'] == 'portuguese']['text_translated'].tolist()

lemmatized_texts = lemmatization(df_list)

data_words = gen_words(lemmatized_texts)

# Bigrams and Trigrams
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

# TF - IDF Removal
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.09
words = []
words_missing_in_tfidf = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=20,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto')

pyLDAvis.enable_notebook()
# Change the R parameter below to change the number of relevant words shown
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds', R=20)
print("Topics: Audio from Brazilian Videos")
vis

Topics: Audio from Brazilian Videos


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.163387  0.332028       1        1  14.527547
17    -0.078032 -0.317467       2        1   8.784794
8      0.243068  0.239400       3        1   8.052725
6     -0.254165 -0.213628       4        1   7.251172
13    -0.342496  0.082719       5        1   6.394832
18     0.060813  0.276959       6        1   5.816518
19     0.253266  0.036839       7        1   5.469199
15    -0.106147 -0.061521       8        1   4.352927
10     0.085632  0.000406       9        1   4.231863
4     -0.130369  0.178890      10        1   4.231038
7     -0.191432 -0.077464      11        1   4.125241
9      0.000988 -0.206264      12        1   4.022774
12    -0.122515  0.069317      13        1   3.983698
16     0.163182 -0.074717      14        1   3.720212
11     0.256641 -0.123892      15        1   3.466201
5      0.136207  0.090678      16        1   2.937068
1      0.024574  0.094174      17        1   2.596903
0      0.165590 -0.231759      18        1   2.250867
2     -0.009208 -0.093360      19        1   2.150072
14     0.007790 -0.001338      20        1   1.634349, topic_info=            Term         Freq        Total Category  logprob  loglift
2417        mind  1351.000000  1351.000000  Default  20.0000  20.0000
138      anxiety   733.000000   733.000000  Default  19.0000  19.0000
2509      people   857.000000   857.000000  Default  18.0000  18.0000
3020      friend   734.000000   734.000000  Default  17.0000  17.0000
649         need   633.000000   633.000000  Default  16.0000  16.0000
...          ...          ...          ...      ...      ...      ...
7059  generalmte     0.777989     1.985120  Topic20  -6.9244   3.1772
7075       mesio     0.777989     1.985120  Topic20  -6.9244   3.1772
7063    giouania     0.777989     1.985120  Topic20  -6.9244   3.1772
2809        like     4.852078   138.825463  Topic20  -5.0939   0.7601
2623       truth     1.184661    22.353409  Topic20  -6.5039   1.1764

[539 rows x 6 columns], token_table=      Topic      Freq                                Term
term                                                     
2542     12  0.986890                             ability
6457      8  0.871236                            academic
2575     18  0.478584                         accelerated
1605      1  0.975387                       activity_less
1687      7  0.922705  adaptation_skeletal_muscle_adipose
...     ...       ...                                 ...
2968     12  0.887878                                wish
2538     17  0.977186                               woman
2346     17  0.992755                                work
3185      3  0.988430                               write
2624      5  0.966699                                 you

[569 rows x 3 columns], R=20, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 18, 9, 7, 14, 19, 20, 16, 11, 5, 8, 10, 13, 17, 12, 6, 2, 1, 3, 15])

In [39]:
# Text topic model for American videos

df_list = total_df[total_df['language'] == 'english']['text_translated'].tolist()

lemmatized_texts = lemmatization(df_list)

data_words = gen_words(lemmatized_texts)

# Bigrams and Trigrams
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

# TF - IDF Removal
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.09
words = []
words_missing_in_tfidf = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=19,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto')

pyLDAvis.enable_notebook()
# Change the R parameter below to change the number of relevant words shown
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds', R=20)
print("Topics: Audio from American Videos")
vis

Topics: Audio from American Videos


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
2     -0.279928 -0.173082       1        1  9.208710
7     -0.097127 -0.306958       2        1  9.007859
6     -0.307238  0.029025       3        1  7.934999
15    -0.205389  0.218096       4        1  6.679298
3      0.044199 -0.206046       5        1  6.317427
11     0.153893 -0.281216       6        1  5.937358
1      0.129084  0.265657       7        1  5.932739
9     -0.017851  0.278496       8        1  5.565562
10    -0.116474  0.136761       9        1  5.214571
14    -0.122104 -0.139999      10        1  5.070431
13     0.186374 -0.127662      11        1  4.929464
12     0.233828 -0.038798      12        1  4.746052
17     0.266774  0.051310      13        1  4.560002
16     0.169213  0.144568      14        1  4.332703
5     -0.155820 -0.006225      15        1  4.045615
0      0.099773  0.081279      16        1  3.676794
4     -0.002279  0.115910      17        1  2.735195
8     -0.024864 -0.019341      18        1  2.195094
18     0.045937 -0.021774      19        1  1.910127, topic_info=              Term        Freq       Total Category  logprob  loglift
388            get  833.000000  833.000000  Default  20.0000  20.0000
1108          tell  595.000000  595.000000  Default  19.0000  19.0000
1161        update  626.000000  626.000000  Default  18.0000  18.0000
1067           say  611.000000  611.000000  Default  17.0000  17.0000
106           feel  560.000000  560.000000  Default  16.0000  16.0000
...            ...         ...         ...      ...      ...      ...
1175           way   60.258161  198.943547  Topic19  -2.9188   2.7636
113   relationship   39.942934  549.299161  Topic19  -3.3300   1.3368
726          thing   37.236757  480.343070  Topic19  -3.4001   1.4008
112        partner   12.616757   92.949796  Topic19  -4.4824   1.9610
1502         bring    8.879461  173.984896  Topic19  -4.8337   0.9828

[587 rows x 6 columns], token_table=      Topic      Freq                                  Term
term                                                       
3495      6  0.885479                                  aauz
3495     13  0.102963                                  aauz
7244      6  0.974892                                  aavz
8108     10  0.846580  aavz_wasabout_wonderfull_problematic
7246      6  0.972173                                  abeh
...     ...       ...                                   ...
3482      1  0.966414                                 wound
4557     15  0.915878                                   wth
6086      3  0.958079                        wth_thelevelof
9804     11  0.950474                        yesterday_live
4304      9  0.801053                                 youth

[675 rows x 3 columns], R=20, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 8, 7, 16, 4, 12, 2, 10, 11, 15, 14, 13, 18, 17, 6, 1, 5, 9, 19])